In [ ]:
%load_ext autoreload
%autoreload 2

Description:

Ce notebook est utile pour afficher les courbes de tendance de SoH par modèle (pour l'instant tesla) et on peut même aller plus loin avec les courbe de tendance de SoH pour les versions (ou tesla code).

Il y'a la possibilité d'ajouter les intervals de confiance de ces courbes. 

In [ ]:
import pandas as pd
from core.sql_utils import *
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

from transform.insights_results.trendlines import *
from transform.insights_results.trendlines_excel import *


In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT *  FROM  vehicle v	
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join vehicle_data vd 
            on vd.vehicle_id  = v.id
            join battery b 
            on b.id = vm.battery_id""")
            #where o.id in ('5b89ebf7-8774-41ec-97f8-7dd4e8a2fd0f');""")
                             , con)



dbeaver_df.head()
dbeaver_df['timestamp'] = pd.to_datetime(dbeaver_df['timestamp'])
dbeaver_df.sort_values("timestamp", inplace=True)

### Fonction plot

In [ ]:
def trendline_apply(x, f):
    return eval(f)

def graph(df, trendline, trendline_max, trendline_min, model):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df['odometer'],
        y=df['soh'],
        mode='markers',
        marker_color='rgba(50, 182, 193, .9)',
        name='SoH compute'
    ))

    x_sorted = df['odometer'].sort_values()
    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline['trendline']),
        mode='lines',
        line=dict(color='red'),
        name='Fit'
    ))

    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline_max['trendline']),
        mode='lines',
        line=dict(color='green'),
        name='Upper'
    ))

    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline_min['trendline']),
        mode='lines',
        line=dict(color='green'),
        name='Lower'
    ))

    fig.update_layout(
        width=1000,
        height=600,
        xaxis_title='Odometer',
        yaxis_title='State of Health (SoH)',
        legend_title='Légende',
        title=f"model: {model}",
        template='plotly_white'
    )

    return fig.show()
    

## Graphs tendances pour les modèles

In [ ]:
for model in dbeaver_df['model_name'].unique():
    df_model = dbeaver_df[dbeaver_df['model_name'] == model].dropna(subset='soh')
    if model in ["mistang mach-e", "midel 3", "taycian", "unknown", "zoey"]:
        pass
    elif df_model.shape[0] > 10:
            trendline, trendline_max, trendline_min = get_trendlines(df_model)
            
            graph(df_model, trendline, trendline_max, trendline_min, model)
            
    else:
        print(f"Only {df_model.shape[0]} SoH available for {model} not enougth to compute trendline")

## Graphs des tendances pour les types/versions

In [ ]:
for model in dbeaver_df['model_name'].unique():
    if model in ["mistang mach-e", "midel 3", "taycian", "unknown", "zoey"]:
        pass
    elif model in ["model 3", "model y", "model x", "model s"]:
        for version in dbeaver_df['version'].unique():
            df_model = dbeaver_df[dbeaver_df['version'] == version].dropna(subset='soh')
            if df_model.shape[0] > 10:
                trendline, trendline_max, trendline_min = get_trendlines(df_model)
                graph(df_model, trendline, trendline_max, trendline_min, version)
            else:
                print(f"Only {df_model.shape[0]} SoH available for {version} not enougth to compute trendline")
    else:
        for type_ in dbeaver_df['type'].unique():
            df_model = dbeaver_df[dbeaver_df['type'] == type_].dropna(subset='soh')
            if df_model.shape[0] > 10:
                trendline, trendline_max, trendline_min = get_trendlines(df_model)
                graph(df_model, trendline, trendline_max, trendline_min, type_)
            else:
                print(f"Only {df_model.shape[0]} SoH available for {type_} not enougth to compute trendline")

## Graph GSheet

In [ ]:
client = get_gspread_client()
df_sheet = load_excel_data(client)

In [ ]:
for model in df_sheet["Modèle"].unique():
    df_model = df_sheet[df_sheet['Modèle']==model]
    try:
        trendline, trendline_max, trendline_min = get_trendlines(df_model)
        graph(df_model, trendline, trendline_max, trendline_min, model)
    except Exception as e:
        logging.error(f"Erreur lors du traitement du modèle {model} : {e}")

## Graph for one type/version/model

In [ ]:
## comment what you don't need and change type/model/version or Modèle/Type/OEM
#from dbeaver
df_model = dbeaver_df[dbeaver_df['type']=="your_type"]
# from gsheet
df_model = df_sheet[df_sheet['Modèle']=="your_type"]
trendline, trendline_max, trendline_min = get_trendlines(df_model)
graph(df_model, trendline, trendline_max, trendline_min, model)
